# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib

warnings.filterwarnings(action='ignore')

In [2]:
defender_model = joblib.load('./models/defender_lgbm.pkl')

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
defender_df = premier_df[premier_df['포지션']=='수비수']

In [5]:
num_col = [defender_df.columns[i] for i,x in enumerate(defender_df.dtypes) if x == 'int64' or x == 'float64']
independent_value = set(num_col) - set(['평균획득승점/경기','출장시간','승','무','패','경기당 팀 득점/90','경기당 팀 실점/90'])
dependent_value = '평균획득승점/경기'

In [6]:
defender_df['예측PPG']=defender_model.predict(defender_df.loc[:,independent_value])

In [7]:
defender_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
242,Vladimír Coufal,2021-2022,수비수,웨스트 햄,잉글랜드 프리미어 리그,2,2,0,16,13,...,84,39.67,35.90,1.46,3.06,0.82,6.12,8.76,1.39,1.981833
241,Declan Rice,2021-2022,수비수,웨스트 햄,잉글랜드 프리미어 리그,4,11,0,19,16,...,79,52.53,47.83,0.93,2.02,0.23,3.70,0.70,1.43,1.685289
178,Douglas Luiz,2021-2022,수비수,아스톤 빌라,잉글랜드 프리미어 리그,5,1,0,8,3,...,62,59.83,52.67,1.12,1.33,0.50,3.54,0.75,1.42,1.629825
170,Thomas Partey,2021-2022,수비수,아스날,잉글랜드 프리미어 리그,4,3,0,22,2,...,88,55.15,47.46,1.48,3.22,0.13,4.38,1.04,1.94,1.627230
33,Wilfred Ndidi,2021-2022,수비수,레스터,잉글랜드 프리미어 리그,4,3,0,14,7,...,66,54.10,50.41,0.69,1.58,0.16,4.16,0.63,1.48,1.611637


In [8]:
defender_min = np.min(defender_df['예측PPG'].tolist())

In [9]:
defender_df['WAR'] = defender_df['예측PPG'].map(lambda x : x-defender_min+1)

In [10]:
defender_df.sort_values(by='WAR',ascending=False).head(60)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG,WAR
242,Vladimír Coufal,2021-2022,수비수,웨스트 햄,잉글랜드 프리미어 리그,2,2,0,16,13,...,39.67,35.90,1.46,3.06,0.82,6.12,8.76,1.39,1.981833,1.942875
241,Declan Rice,2021-2022,수비수,웨스트 햄,잉글랜드 프리미어 리그,4,11,0,19,16,...,52.53,47.83,0.93,2.02,0.23,3.70,0.70,1.43,1.685289,1.646330
178,Douglas Luiz,2021-2022,수비수,아스톤 빌라,잉글랜드 프리미어 리그,5,1,0,8,3,...,59.83,52.67,1.12,1.33,0.50,3.54,0.75,1.42,1.629825,1.590867
170,Thomas Partey,2021-2022,수비수,아스날,잉글랜드 프리미어 리그,4,3,0,22,2,...,55.15,47.46,1.48,3.22,0.13,4.38,1.04,1.94,1.627230,1.588271
33,Wilfred Ndidi,2021-2022,수비수,레스터,잉글랜드 프리미어 리그,4,3,0,14,7,...,54.10,50.41,0.69,1.58,0.16,4.16,0.63,1.48,1.611637,1.572679
197,Allan,2021-2022,수비수,에버튼,잉글랜드 프리미어 리그,4,3,0,15,15,...,51.46,46.12,0.94,1.63,0.31,3.26,0.69,1.58,1.575607,1.536649
87,Rodri,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,3,3,0,24,10,...,64.34,60.11,0.79,2.43,0.23,5.82,0.11,1.82,1.572724,1.533766
89,Fernandinho,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,2,3,0,25,7,...,60.05,55.47,0.74,2.45,0.04,4.84,0.20,1.95,1.565242,1.526284
249,Aaron Cresswell,2021-2022,수비수,웨스트 햄,잉글랜드 프리미어 리그,0,5,0,16,12,...,46.31,41.50,1.12,2.45,0.81,4.06,8.12,1.50,1.561702,1.522744
262,Antonio Rüdiger,2021-2022,수비수,첼시,잉글랜드 프리미어 리그,2,1,0,28,7,...,58.72,55.88,0.19,0.74,0.43,10.15,0.34,1.98,1.556654,1.517695


In [11]:
defender_df[['이름','시즌','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,시즌,WAR
242,Vladimír Coufal,2021-2022,1.942875
241,Declan Rice,2021-2022,1.646330
178,Douglas Luiz,2021-2022,1.590867
170,Thomas Partey,2021-2022,1.588271
33,Wilfred Ndidi,2021-2022,1.572679
197,Allan,2021-2022,1.536649
87,Rodri,2021-2022,1.533766
89,Fernandinho,2021-2022,1.526284
249,Aaron Cresswell,2021-2022,1.522744
262,Antonio Rüdiger,2021-2022,1.517695
